# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
import os

import joblib

In [2]:
# config.json downloaded form Azure Envoirment
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

quick-starts-ws-236295
aml-quickstarts-236295
westus2
81cefad3-d2c9-4f77-a466-99a7f541c7bb


In [3]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'heart-faliure-prediction'
project_folder = './capstone-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
heart-faliure-prediction,quick-starts-ws-236295,Link to Azure Machine Learning studio,Link to Documentation


In [4]:
# Choose a name for the cluster
cpu_cluster_name = "demo11"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    print('Creating a new compute cluster...')
    # Poll for a minimum number of nodes (min_nodes = 1). 
    # If no min node count is provided it uses the scale settings for the cluster.
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2', min_nodes=1, max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

The Heart Faliure Dataset is publically available dataset on Kaggle. It consist of data from 299 patients out of which 105 are women and 194 are men with Heart Faliure. The aim of this project is to classify patients based on their odds of survival.

In [24]:
found = False
key = "Heart-Failure-Dataset"
description_text = "Prediction of survival of patients with heart failure"

                                
df = dataset.to_pandas_dataframe()
df.describe()


,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.000000,299.00000,299.000000,299.000000,299.00000,299.000000,299.00000
mean,60.833893,0.431438,581.839465,0.418060,38.083612,0.351171,263358.029264,1.39388,136.625418,0.648829,0.32107,130.260870,0.32107
std,11.894809,0.496107,970.287881,0.494067,11.834841,0.478136,97804.236869,1.03451,4.412477,0.478136,0.46767,77.614208,0.46767
min,40.000000,0.000000,23.000000,0.000000,14.000000,0.000000,25100.000000,0.50000,113.000000,0.000000,0.00000,4.000000,0.00000
25%,51.000000,0.000000,116.500000,0.000000,30.000000,0.000000,212500.000000,0.90000,134.000000,0.000000,0.00000,73.000000,0.00000
50%,60.000000,0.000000,250.000000,0.000000,38.000000,0.000000,262000.000000,1.10000,137.000000,1.000000,0.00000,115.000000,0.00000
75%,70.000000,1.000000,582.000000,1.000000,45.000000,1.000000,303500.000000,1.40000,140.000000,1.000000,1.00000,203.000000,1.00000
max,95.000000,1.000000,7861.000000,1.000000,80.000000,1.000000,850000.000000,9.40000,148.000000,1.000000,1.00000,285.000000,1.00000


## Hyperdrive Configuration

The Hyperparameters of the Logistic Regression Model from SciKit-Learn are optimized in this step using HyperDrive.
The brief detail of each steps and Hyperparameter used is as follow:

### Early Stopping Policy
It improve the computational efficiency of the run by termination if is running poorly. The BanditPolicy is used as Early Stopping Policy.

### Parameter Sampler (ps)
The parameter sampler has discrete values for C (regularization) and max_iter (maximum iterations). I chose RandomParameterSampling for speed and early termination. However, if budget allows, GridParameterSampling or BayesianParameterSampling would provide more thorough exploration of the hyperparameter space.

### HyperDriveConfig

The chosen configuration for HyperDrive is as follows:

##### hyperparameter_sampling
Specifies the hyperparameter sampling space. 

##### primary_metric_name
The name of the primary metric reported by the experiment runs, which in this case is "Accuracy". 

##### primary_metric_goal 
Set to PrimaryMetricGoal.MAXIMIZE, indicating that the primary metric should be maximized during evaluation. 

##### policy
Refers to the early termination policy that has been specified. 

##### estimator
An estimator that will be used with the sampled hyperparameters. In this case, the estimator option was chosen, while the other two options, run_config and pipeline, were not selected. The estimator will be used in conjunction with the "train.py" file, which performs basic data manipulation. 

##### max_total_runs
The maximum total number of runs to create. While 16 is set as the upper bound, the actual number of runs may be lower if the sample space is smaller. If both "max_total_runs" and "max_duration_minutes" are provided, the hyperparameter tuning experiment will terminate when either of these thresholds is reached. 

##### max_concurrent_runs
Specifies the maximum number of runs to execute concurrently. If set to None, all runs are launched in parallel. The number of concurrent runs depends on the available resources in the specified compute target, so it is important to ensure that the compute target has sufficient resources for the desired concurrency.



In [6]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

#TODO: Create the different params that you will be using during training
ps = RandomParameterSampling(
    {
        '--C' : choice(0.001,0.01,0.1,1,10,20,50,100,200),
        '--max_iter': choice(50,100,200,300)
    }
)

if "training" not in os.listdir():
    os.mkdir("./training")

#TODO: Create your estimator and hyperdrive config
# Created a SKLearn estimator for use with train.py
est =SKLearn(source_directory = "./", compute_target=compute_target, vm_size='STANDARD_DS3_V2', 
                   entry_script="train.py")

hyperdrive_run_config = HyperDriveConfig(hyperparameter_sampling=ps, primary_metric_name='Accuracy', 
                                         primary_metric_goal=PrimaryMetricGoal.MAXIMIZE, policy=policy, 
                                         estimator=est, max_total_runs=16, max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [7]:
from azureml.core.experiment import Experiment

# Submit the hyperdrive run to the experiment and show run details with the widget.

# Start the HyperDrive run
hyperdrive_run = experiment.submit(hyperdrive_run_config)

# Monitor HyperDrive runs 
# You can monitor the progress of the runs with the following Jupyter widget
RunDetails(hyperdrive_run).show()

hyperdrive_run.wait_for_completion(show_output=True)

assert(hyperdrive_run.get_status() == "Completed")


_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_fb76fc18-6db7-4529-8d27-692fc33c0c19
Web View: https://ml.azure.com/runs/HD_fb76fc18-6db7-4529-8d27-692fc33c0c19?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-236295/workspaces/quick-starts-ws-236295&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-06-21T22:15:18.823692][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-06-21T22:15:19.2449624Z][SCHEDULER][INFO]Scheduling job, id='HD_fb76fc18-6db7-4529-8d27-692fc33c0c19_0' 
[2023-06-21T22:15:19.3414065Z][SCHEDULER][INFO]Scheduling job, id='HD_fb76fc18-6db7-4529-8d27-692fc33c0c19_1' 
[2023-06-21T22:15:19.3978278Z][SCHEDULER][INFO]Scheduling job, id='HD_fb76fc18-6db7-4529-8d27-692fc33c0c19_2' 
[2023-06-21T22:15:19.5148507Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_fb76fc18-6db7-4529-8d27-692fc33c0c19_0' 
[2023-06-21T22:15:19.489921][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to t

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [15]:
hyperdrive_best_run = hyperdrive_run.get_best_run_by_primary_metric() 

In [25]:
print('Best Run Id: ', hyperdrive_best_run.id) 


Best Run Id:  HD_fb76fc18-6db7-4529-8d27-692fc33c0c19_2


In [26]:
# get_metrics()
# Returns the metrics
print("Best run metrics :",hyperdrive_best_run.get_metrics())
print("Best run metrics :",hyperdrive_best_run.get_metrics()['Accuracy'])


Best run metrics : {'Regularization Strength:': 100.0, 'Accuracy': 0.8333333333333334, 'Max iterations:': 50}
Best run metrics : 0.8333333333333334


In [27]:
# get_details()
# Returns a dictionary with the details for the run
print("Best run details :",hyperdrive_best_run.get_details() ['runDefinition']['arguments'])


Best run details : ['--C', '100', '--max_iter', '50']


In [28]:
# get_properties()
# Fetch the latest properties of the run from the service
print("Best run properties :",hyperdrive_best_run.get_properties())


Best run properties : {'_azureml.ComputeTargetType': 'amlctrain', 'ContentSnapshotId': '714ecd87-9355-4b76-90c3-87a57d220f91', 'ProcessInfoFile': 'azureml-logs/process_info.json', 'ProcessStatusFile': 'azureml-logs/process_status.json'}


In [22]:
hyperdrive_best_run


Experiment,Id,Type,Status,Details Page,Docs Page
heart-faliure-prediction,HD_fb76fc18-6db7-4529-8d27-692fc33c0c19_2,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [20]:
#TODO: Save the best model
# Save the best model
hyperdrive_best_run.register_model(model_name = "HyperDrive_best_model", model_path = './outputs/')
print("Completed")

Completed


In [21]:
# Download the model file

hyperdrive_best_run.download_file('outputs/model.pkl', 'HyperDrive_best_model.pkl')

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

